In [1]:
import multiprocessing
import sys
from pettingzoo.test import parallel_api_test

import multiprocessing
import sys
from pettingzoo.test import parallel_api_test
from citylearn import GridLearn
from citylearn import MyEnv
from pathlib import Path
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3 import PPO
import gym

import multiprocessing
import sys
import supersuit as ss

import time

# multiprocessing.set_start_method("fork")

climate_zone = 1
data_path = Path("../citylearn/data/Climate_Zone_"+str(climate_zone))
buildings_states_actions = '../citylearn/buildings_state_action_space.json'

config = {
    "data_path":data_path,
    "climate_zone":climate_zone,
    "buildings_states_actions_file":buildings_states_actions,
    "hourly_timesteps":4
}

grid = GridLearn(**config)

envs = [MyEnv(grid), MyEnv(grid)]

print('padding action/observation spaces...')
envs = [ss.pad_action_space_v0(env) for env in envs]
envs = [ss.pad_observations_v0(env) for env in envs]

print('creating pettingzoo env...')
envs = [ss.pettingzoo_env_to_vec_env_v0(env) for env in envs]

print('stacking vec env...')
nenvs = 2
envs = [ss.concat_vec_envs_v0(env, nenvs, num_cpus=1, base_class='stable_baselines3') for env in envs]

../citylearn/buildings_state_action_space.json


/Users/aislingpigott/Documents/CityLearn/citylearn/energy_models.py:516: RuntimeWarning: divide by zero encountered in true_divide
  self.cooling_device.cop_cooling = self.cooling_device.eta_tech*(np.add(self.cooling_device.t_target_cooling,273.15))/np.subtract(self.sim_results['t_out'],self.cooling_device.t_target_cooling)
/Users/aislingpigott/Documents/CityLearn/citylearn/energy_models.py:516: RuntimeWarning: divide by zero encountered in true_divide
  self.cooling_device.cop_cooling = self.cooling_device.eta_tech*(np.add(self.cooling_device.t_target_cooling,273.15))/np.subtract(self.sim_results['t_out'],self.cooling_device.t_target_cooling)
/Users/aislingpigott/Documents/CityLearn/citylearn/energy_models.py:516: RuntimeWarning: divide by zero encountered in true_divide
  self.cooling_device.cop_cooling = self.cooling_device.eta_tech*(np.add(self.cooling_device.t_target_cooling,273.15))/np.subtract(self.sim_results['t_out'],self.cooling_device.t_target_cooling)
/Users/aislingpigott/D

padding action/observation spaces...
creating pettingzoo env...
stacking vec env...


In [2]:
print('setting the grid...')
for env in envs[1:]:
    for n in range(nenvs):
        env.venv.vec_envs[n].par_env.aec_env.env.env.env.env.grid = envs[0].venv.vec_envs[n].par_env.aec_env.env.env.env.env.grid

setting the grid...


In [3]:
models = [PPO(MlpPolicy, env, verbose=2, gamma=0.999, batch_size=2, n_steps=1, ent_coef=0.01, learning_rate=0.00025, vf_coef=0.5, max_grad_norm=0.5, gae_lambda=0.95, n_epochs=4, clip_range=0.2, clip_range_vf=1) for env in envs]

Using cpu device
Using cpu device


In [4]:
import numpy as np
obss = [env.reset() for env in envs]
for _ in range(5):
    for m in range(len(models)):
        obss[m] = [np.concatenate(list(envs[m].venv.vec_envs[i].par_env.state().values())) for i in range(len(envs[m].venv.vec_envs))]
        obss[m] = np.concatenate(obss[m])
        obss[m] = obss[m].reshape(2*nenvs,-1)

        action = models[m].predict(obss[m])[0]
        obss[m], reward, done, info = envs[m].step(action)
        print(obss[m].shape)

calling reset...
['88CD4', '24SN7']
calling reset...
['88CD4', '24SN7']
calling reset...
['JHBY9', 'JBAQL']
calling reset...
['JHBY9', 'JBAQL']
['88CD4', '24SN7']
['88CD4', '24SN7']
<class 'supersuit.aec_wrappers.pad_observations'>
<class 'supersuit.aec_wrappers.pad_observations'>
['88CD4', '24SN7'] dict_keys(['88CD4', '24SN7'])
0
['88CD4', '24SN7']
    name      p_mw
0  88CD4  0.028037
1  JHBY9  0.000000
2  24SN7  0.008406
3  JBAQL  0.000000
<class 'supersuit.aec_wrappers.pad_observations'>
<class 'supersuit.aec_wrappers.pad_observations'>
['88CD4', '24SN7'] dict_keys(['88CD4', '24SN7'])
0
['88CD4', '24SN7']
    name      p_mw
0  88CD4  0.007876
1  JHBY9  0.000000
2  24SN7  0.008119
3  JBAQL  0.000000
(4, 29)
['JHBY9', 'JBAQL']
['JHBY9', 'JBAQL']
<class 'supersuit.aec_wrappers.pad_observations'>
<class 'supersuit.aec_wrappers.pad_observations'>
['JHBY9', 'JBAQL'] dict_keys(['JHBY9', 'JBAQL'])
0
['JHBY9', 'JBAQL']
    name      p_mw
0  88CD4  0.028037
1  JHBY9  0.007592
2  24SN7  0.008

In [5]:
for _ in range(5):
    for model in models:
        model.learn(1, reset_num_timesteps=False)

calling reset...
['88CD4', '24SN7']
calling reset...
['88CD4', '24SN7']
<class 'supersuit.aec_wrappers.pad_observations'>
<class 'supersuit.aec_wrappers.pad_observations'>
['88CD4', '24SN7'] dict_keys(['88CD4', '24SN7'])
5
['88CD4', '24SN7']
    name      p_mw
0  88CD4  0.044615
1  JHBY9  0.016841
2  24SN7  0.045955
3  JBAQL  0.009765
<class 'supersuit.aec_wrappers.pad_observations'>
<class 'supersuit.aec_wrappers.pad_observations'>
['88CD4', '24SN7'] dict_keys(['88CD4', '24SN7'])
5
['88CD4', '24SN7']
    name      p_mw
0  88CD4  0.006380
1  JHBY9  0.007176
2  24SN7  0.006916
3  JBAQL  0.007306
---------------------------
| time/              |    |
|    fps             | 30 |
|    iterations      | 1  |
|    time_elapsed    | 0  |
|    total_timesteps | 4  |
---------------------------
calling reset...
['JHBY9', 'JBAQL']
calling reset...
['JHBY9', 'JBAQL']
<class 'supersuit.aec_wrappers.pad_observations'>
<class 'supersuit.aec_wrappers.pad_observations'>
['JHBY9', 'JBAQL'] dict_keys([

<class 'supersuit.aec_wrappers.pad_observations'>
<class 'supersuit.aec_wrappers.pad_observations'>
['JHBY9', 'JBAQL'] dict_keys(['JHBY9', 'JBAQL'])
8
['JHBY9', 'JBAQL']
    name      p_mw
0  88CD4  0.010781
1  JHBY9  0.007495
2  24SN7  0.009862
3  JBAQL  0.007649
<class 'supersuit.aec_wrappers.pad_observations'>
<class 'supersuit.aec_wrappers.pad_observations'>
['JHBY9', 'JBAQL'] dict_keys(['JHBY9', 'JBAQL'])
8
['JHBY9', 'JBAQL']
    name      p_mw
0  88CD4  0.010447
1  JHBY9  0.006943
2  24SN7  0.045623
3  JBAQL  0.006578
-----------------------------------------
| time/                   |             |
|    fps                  | 192         |
|    iterations           | 1           |
|    time_elapsed         | 0           |
|    total_timesteps      | 16          |
| train/                  |             |
|    approx_kl            | 0.008369565 |
|    clip_fraction        | 0           |
|    clip_range           | 0.2         |
|    clip_range_vf        | 1           |
|    ent